In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import os
from PIL import Image
try:
    import torchinfo
except:
    !pip install torchinfo
    import torchinfo
from torchinfo import summary
from tqdm.auto import tqdm
from timeit import default_timer as timer
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import datasets
import matplotlib.pyplot as plt
from typing import Tuple, Dict, List




device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [2]:
nat_4ml_folder_path = "/content/drive/MyDrive/Tube_Vision Model/Raw/4ml_unique"
lab1_folder_path = "/content/drive/MyDrive/Tube_Vision Model/Raw/labor_1_unique"

loaded_raw_4ml_nat = [Image.open(os.path.join(nat_4ml_folder_path, f)) for f in os.listdir(nat_4ml_folder_path)]
loaded_raw_lab1 = [Image.open(os.path.join(lab1_folder_path, f)) for f in os.listdir(lab1_folder_path)]

In [ ]:
# DO NOT RUN!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
from sklearn.model_selection import train_test_split
import shutil

def separate_data_into_train_test(x):
  '''
  0 = nat
  1 = lab1
  '''


  if x == 0:
    train_dir = "/content/drive/MyDrive/Tube_Vision Model/nat4ml_lab1/train/nat_4ml"
    test_dir = "/content/drive/MyDrive/Tube_Vision Model/nat4ml_lab1/test/nat_4ml"

    nat_train, nat_test = train_test_split(loaded_raw_4ml_nat,
                                           test_size=0.4,
                                           random_state=777)
    print("Number of images in nat_train:", len(nat_train))
    print("Number of images in nat_test:", len(nat_test))



    for img in nat_train:
      img_path = img.filename
      filename = os.path.basename(img.filename)
      new_image_path = os.path.join(train_dir, filename)
      shutil.copyfile(img_path, new_image_path)
    print(len(os.listdir(train_dir)))

    for img in nat_test:
      img_path = img.filename
      filename = os.path.basename(img.filename)
      new_image_path = os.path.join(test_dir, filename)
      shutil.copyfile(img_path, new_image_path)
    print(len(os.listdir(test_dir)))

  elif x == 1:
    train_dir = "/content/drive/MyDrive/Tube_Vision Model/nat4ml_lab1/train/lab1"
    test_dir = "/content/drive/MyDrive/Tube_Vision Model/nat4ml_lab1/test/lab1"

    lab1_train, lab1_test = train_test_split(loaded_raw_lab1,
                                           test_size=0.4,
                                           random_state=777)
    print("Number of images in lab1_train:", len(lab1_train))
    print("Number of images in lab1_test:", len(lab1_test))

    for img in lab1_train:
      img_path = img.filename
      filename = os.path.basename(img.filename)
      new_image_path = os.path.join(train_dir, filename)
      shutil.copyfile(img_path, new_image_path)
    print(len(os.listdir(train_dir)))

    for img in lab1_test:
      img_path = img.filename
      filename = os.path.basename(img.filename)
      new_image_path = os.path.join(test_dir, filename)
      shutil.copyfile(img_path, new_image_path)
    print(len(os.listdir(test_dir)))



separate_data_into_train_test(0)
separate_data_into_train_test(1)

Number of images in nat_train: 61
Number of images in nat_test: 41
61
41
Number of images in lab1_train: 33
Number of images in lab1_test: 23
33
23


In [ ]:
# ALSO DO NOT RUNN
import time

def empty_train_test_subclasses():
  """
  Deletes every image from train/test image folders.
  """
  folder_list = [
      "/content/drive/MyDrive/Tube_Vision Model/nat4ml_lab1/train/lab1",
      "/content/drive/MyDrive/Tube_Vision Model/nat4ml_lab1/train/nat_4ml",
      "/content/drive/MyDrive/Tube_Vision Model/nat4ml_lab1/test/lab1",
      "/content/drive/MyDrive/Tube_Vision Model/nat4ml_lab1/test/nat_4ml"
  ]

  for folder_path in folder_list:
      files = os.listdir(folder_path)
      for file in files:
          file_path = os.path.join(folder_path, file)
          os.remove(file_path)

  time.sleep(20)

empty_train_test_subclasses()

In [3]:
train_dir ="/content/drive/MyDrive/Tube_Vision Model/nat4ml_lab1/train"
test_dir ="/content/drive/MyDrive/Tube_Vision Model/nat4ml_lab1/test"

In [4]:
augmented_transform = transforms.Compose([
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    transforms.ToTensor()
])

simple_transform = transforms.Compose([
    transforms.ToTensor()
])

train_dir ="/content/drive/MyDrive/Tube_Vision Model/nat4ml_lab1/train"
test_dir ="/content/drive/MyDrive/Tube_Vision Model/nat4ml_lab1/test"

# Load dataset
train_dataset = datasets.ImageFolder(root=train_dir,
                                     transform=augmented_transform)
test_dataset = datasets.ImageFolder(root=test_dir,
                                    transform=simple_transform)

# Create data loaders
train_loader = DataLoader(train_dataset,
                          batch_size=1,
                          num_workers=2)
test_loader = DataLoader(test_dataset,
                         batch_size=1,
                         num_workers=2)

# Define ResNet model
model_resnet = models.resnet18(weights=None)
num_features = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_features, len(train_dataset.classes))

# Move the model to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_resnet = model_resnet.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_resnet.parameters(), lr=0.001)

# Training loop
for epoch in range(5):  # Number of epochs
    model_resnet.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_resnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation loop
    model_resnet.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model_resnet(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            val_loss += criterion(outputs, labels).item()

    print(f"Epoch {epoch + 1}:")
    print(f"Validation Loss: {val_loss / len(test_loader)}")
    print(f"Validation Accuracy: {100 * correct / total}%")

Epoch 1:
Validation Loss: 5.103132523429881
Validation Accuracy: 64.0625%
Epoch 2:
Validation Loss: 3.8891409055725354
Validation Accuracy: 64.0625%
Epoch 3:
Validation Loss: 0.7543516487348825
Validation Accuracy: 64.0625%
Epoch 4:
Validation Loss: 1.6646219156382998
Validation Accuracy: 64.0625%
Epoch 5:
Validation Loss: 2.1232724981455244
Validation Accuracy: 64.0625%
